# OpenAI API 사용 RAG (검색 증강 생성)
: https://www.promptingguide.ai/kr/techniques/rag

문서를 벡터로 변환(임베딩) : 텍스트 데이터를 고차원 벡터 공간으로 매핑하는 작업

In [1]:
import os
from dotenv import load_dotenv

# .env 파일의 내용 불러오기
load_dotenv("C:/env/.env")

# 환경 변수 가져오기
API_KEY = os.getenv("OPENAI_API_KEY")
# print(API_KEY)

from openai import OpenAI
client = OpenAI(api_key=API_KEY)

In [7]:
# ! pip install faiss-cpu
import faiss
import numpy as np

# 샘플 문서데이터 (텍스트)
documents = [
    "OpenAI는 인공지능 연구소입니다.",
    "FAISS는 Facebook AI에서 개발한 벡터 검색 라이브러리입니다.",
    "GPT는 자연어 처리를 위한 강력한 AI 모델입니다.",
    "RAG는 검색 기반 생성 기법을 활용하여 AI 응답을 향상시키는 방법입니다."    
]

In [3]:
# 문서를 벡터로 변환 (임베딩)
def get_embedding(text):
    response = client.embeddings.create(
        model="text-embedding-ada-002",
        input=text
    )
    return np.array(response.data[0].embedding)

# 모든 문서의 임베딩 생성
document_embeddings = np.array([get_embedding(doc) for doc in documents])
print(document_embeddings)

# FAISS 벡터 인덱스 생성
dimension = document_embeddings.shape[1]
print(dimension)  # 1536
index = faiss.IndexFlatL2(dimension)
index.add(document_embeddings)

[[-0.00690352 -0.03443271 -0.01008424 ... -0.01365029  0.00289007
   0.00851021]
 [-0.03228273  0.00873255 -0.00940116 ...  0.00610198 -0.00356596
  -0.01077216]
 [ 0.00221576 -0.01288273  0.00589113 ... -0.00712339 -0.0018764
   0.0078812 ]
 [-0.02464437 -0.01466453  0.01109461 ... -0.01292602 -0.00390501
   0.01446546]]
1536


### FAISS (Facebook AI Similarity Search)
Facebook AI 팀에서 개발한 대규모 벡터 검색 라이브러리로, 빠르고 효율적인 최근접 이웃 검색(Nearest Neighbor Search, NNS) 을 수행하는 데 사용된다. 
<br>
특히, 고차원 벡터 검색을 최적화하여 딥러닝 임베딩 검색, 추천 시스템, RAG(Retrieval-Augmented Generation) 등에서 널리 활용된다

In [4]:
#  사용자 질문 입력 및 벡터 변환
query = "RAG의 핵심 구성 요소는 무엇인가?"
query_embedding = get_embedding(query)
print(query_embedding)

#  FAISS를 이용한 검색( 가장 유사한 문서 찾기)
k = 2   # 가장 유사한 2개의 문서 검색
distances, indices = index.search(query_embedding.reshape(1, -1), k)
print(distances)
print(indices)

# 검색된 문서 내용 추출
retrieved_docs = [documents[i] for i in indices[0]]
print(retrieved_docs)

[ 0.00504444 -0.00364638  0.00797835 ... -0.02336373 -0.00101326
  0.015298  ]
[[0.24825913 0.4681806 ]]
[[3 1]]
['RAG는 검색 기반 생성 기법을 활용하여 AI 응답을 향상시키는 방법입니다.', 'FAISS는 Facebook AI에서 개발한 벡터 검색 라이브러리입니다.']


In [5]:
# LLM을 활용하여 최종 응답 생성
context = "\n".join(retrieved_docs)
prompt = f"질문: {query}\n\n참고자료:\n{context}\n\n답변:"
print(prompt)
response = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "system", "content": "당신은 유용한 AI 어시스턴트입니다."},
              {"role": "user", "content": prompt}]
)

# 최종 응답 출력
final_answer = response.choices[0].message.content
print("🔹 AI 응답:", final_answer)

질문: RAG의 핵심 구성 요소는 무엇인가?

참고자료:
RAG는 검색 기반 생성 기법을 활용하여 AI 응답을 향상시키는 방법입니다.
FAISS는 Facebook AI에서 개발한 벡터 검색 라이브러리입니다.

답변:
🔹 AI 응답: RAG의 핵심 구성 요소는 검색 기반 생성 기법입니다. 이 방법은 AI의 응답을 향상시키는데 사용됩니다. 또한 Facebook AI에서 개발한 벡터 검색 라이브러리인 FAISS도 RAG의 중요한 구성 요소로 사용될 수 있습니다.
